In [6]:
import json
from time import sleep
from kafka import KafkaProducer

In [7]:
APIstucture = {
    'keywords': 'python,data,science,analysis',
    'location': 'london',
    'permanent': None,
    'contract': None,
    'temp': None,
    'partTime': None,
    'fullTime': None,
    'minimumSalary': None,
    'maximumSalary': None,
    'postedByRecruitmentAgency': None,
    'postedByDirectEmployer': None,
    'graduate': None,
    'resultsToTake': 100,
    'resultsToSkip': 0
}
def getAPIrequestPages(take=100, skip=0):
    APIstuctureCopy = APIstucture.copy()
    APIstuctureCopy['resultsToTake'] = take
    APIstuctureCopy['resultsToSkip'] = skip
    
    searchParamsString = ''.join([f'{k.lower()}={v}&' for k, v in APIstuctureCopy.items() if v is not None])[:-1]

    return f"https://www.reed.co.uk/api/1.0/search?{searchParamsString}"

In [8]:
import requests
from requests.auth import HTTPBasicAuth

APIkey = '175d9360-1c71-48cb-95ec-0ba958c29a35'

jobListings = []

for i in range(10):
    apiURL = getAPIrequestPages(skip=101*i)
    response = response = requests.get(apiURL, auth=HTTPBasicAuth(APIkey, ''))
    data = response.json()['results']
    jobListings += data

In [ ]:
if __name__ == "__main__":
    SERVER = "broker:9092"

    producer = KafkaProducer(
        bootstrap_servers=[SERVER],
        value_serializer=lambda x: json.dumps(x).encode("utf-8"),
        api_version=(3, 7, 0),
    )   
    
    try:
        i = 0
        while True:
            producer.send("streaming", value=jobListings[i])
            i += 1
            sleep(2)
    except:
        producer.close()